1. Load the CSV file into a DataFrame.
2. Parse the "Radiomics" column, as it contains JSON data.
3. Remove columns with the same values across all rows.

In [81]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
import os

# create random seed for reproducibility
ran_seed = 42

# Load the data from DF_Radiomics_noduls_with_diagnose.csv
file_path = "DF_Radiomics_noduls_with_diagnose.csv"
data = pd.read_csv(file_path)


# Convert the 'Labels' column to an integer
data['Labels'] = data['Labels'].astype(int)

# drop all rows where the label == 0
data = data[data.Labels != 0]

# Parse the JSON in the 'Radiomics' column
data['Radiomics'] = data['Radiomics'].apply(json.loads)

# Convert the 'Radiomics' column into separate columns
radiomics_data = pd.json_normalize(data['Radiomics'])


# Drop the original 'Radiomics' column
data = data.drop('Radiomics', axis=1)


# Reset the indices of both DataFrames
data = data.reset_index(drop=True)
radiomics_data = radiomics_data.reset_index(drop=True)

# Combine the data with the new radiomics columns
data = pd.concat([data, radiomics_data], axis=1)

# Remove columns with the same value across all rows
data = data.loc[:, (data != data.iloc[0]).any()]

#remove columns with all NaN values
data = data.dropna(axis=1, how='all')

print(data.shape)


(309, 103)


In [82]:
data.head()

,Patient,Node,Labels,diagnostics_Image-original_Hash,diagnostics_Image-original_Spacing,diagnostics_Image-original_Size,diagnostics_Image-original_Mean,diagnostics_Image-original_Minimum,diagnostics_Image-original_Maximum,diagnostics_Mask-original_Hash,...,original_gldm_GrayLevelNonUniformity,original_gldm_GrayLevelVariance,original_gldm_HighGrayLevelEmphasis,original_gldm_LargeDependenceEmphasis,original_gldm_LargeDependenceHighGrayLevelEmphasis,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis
0,LIDC-IDRI-0068,Node_N1,3,bea2c9750ea59a0bebb6d3bd63ffacc40fcf6a28,"[0.683594, 0.683594, 1.25]","[512, 512, 261]",-1026.065264,-3024.0,3071.0,0506d1d0d6522eddd1640c8ea75c2fc5a9266270,...,7.355556,60.706173,469.644444,23.444444,16578.377778,0.053875,0.021012,0.488461,152.929922,0.019809
1,LIDC-IDRI-0068,Node_N1,3,bea2c9750ea59a0bebb6d3bd63ffacc40fcf6a28,"[0.683594, 0.683594, 1.25]","[512, 512, 261]",-1026.065264,-3024.0,3071.0,9d7da356d43e2f7ad7f374f6c193e97f6088d7c7,...,7.467153,72.801002,471.051095,17.496350,13573.328467,0.110650,0.024328,0.494688,165.356306,0.010062
2,LIDC-IDRI-0068,Node_N1,3,bea2c9750ea59a0bebb6d3bd63ffacc40fcf6a28,"[0.683594, 0.683594, 1.25]","[512, 512, 261]",-1026.065264,-3024.0,3071.0,c0a43747a23d26b107e21614525f2fd8870ffefc,...,7.685185,43.527006,277.787037,20.370370,9310.490741,0.084481,0.031811,0.463956,84.174037,0.027819
3,LIDC-IDRI-0068,Node_N1,3,bea2c9750ea59a0bebb6d3bd63ffacc40fcf6a28,"[0.683594, 0.683594, 1.25]","[512, 512, 261]",-1026.065264,-3024.0,3071.0,72a09dc3f5d5d146b13402b8ef109422cc3f38a5,...,6.780220,35.367709,229.219780,18.780220,7065.923077,0.084783,0.026368,0.465301,67.725183,0.021973
4,LIDC-IDRI-0072,Node_N1,1,54705f26f9320581c90452445aa820fe9630d5e9,"[0.732422, 0.732422, 1.25]","[512, 512, 305]",-871.936330,-3024.0,3071.0,05efcefff38c73903c3d7839bb987a49176f6068,...,629.334146,45.147393,1253.131545,28.918031,43475.541623,0.020967,0.001319,0.262518,254.476429,0.000632


In [83]:
#remove hash columns
data = data.drop(['diagnostics_Image-original_Hash', 'diagnostics_Mask-original_Hash'], axis=1)

# ok looks like all the objeckt columns except of "Patient" & "Node" are in this form [0.683594, 0.683594, 1.25] which is a list of multiple floats
# exploade them into multiple columns

object_columns = data.select_dtypes(include=['object']).columns.tolist()

# Remove 'Patient' and 'Node' from the list
object_columns.remove('Patient')
object_columns.remove('Node')

# Explode the lists in each object column into multiple columns
for column in object_columns:
    # Convert each list to a Series and expand it into multiple columns
    expanded_columns = data[column].apply(pd.Series)
    
    # Rename the expanded columns to have the original column name as a prefix
    expanded_columns = expanded_columns.rename(columns=lambda x: f"{column}_{x}")
    
    # Drop the original column from the DataFrame
    data = data.drop(column, axis=1)
    
    # Concatenate the expanded columns to the DataFrame
    data = pd.concat([data, expanded_columns], axis=1)

In [84]:
# Create a stratified split
train_data, test_data = train_test_split(data, test_size=0.2, stratify=data['Labels'], random_state=ran_seed)

# if the files already exist, skip this step
if os.path.isfile('DF_Radiomics_noduls_with_diagnose_train_data.csv') and os.path.isfile('DF_Radiomics_noduls_with_diagnose_test_data.csv'):
    print("Files already exist, skipping this step")
else:
    # Save the data to CSV files
    train_data.to_csv('DF_Radiomics_noduls_with_diagnose_train_data.csv', index=False)
    test_data.to_csv('DF_Radiomics_noduls_with_diagnose_test_data.csv', index=False)

Files already exist, skipping this step


In [85]:
print("Train data:", train_data.shape)
print("Test data:", test_data.shape)

Train data: (247, 118)
Test data: (62, 118)


In [86]:
train_data.head()

,Patient,Node,Labels,diagnostics_Image-original_Mean,diagnostics_Image-original_Minimum,diagnostics_Image-original_Maximum,diagnostics_Mask-original_VoxelNum,diagnostics_Mask-original_VolumeNum,original_firstorder_10Percentile,original_firstorder_90Percentile,...,diagnostics_Mask-original_BoundingBox_2,diagnostics_Mask-original_BoundingBox_3,diagnostics_Mask-original_BoundingBox_4,diagnostics_Mask-original_BoundingBox_5,diagnostics_Mask-original_CenterOfMassIndex_0,diagnostics_Mask-original_CenterOfMassIndex_1,diagnostics_Mask-original_CenterOfMassIndex_2,diagnostics_Mask-original_CenterOfMass_0,diagnostics_Mask-original_CenterOfMass_1,diagnostics_Mask-original_CenterOfMass_2
23,LIDC-IDRI-0137,Node_N1,3,-671.885608,-2048.0,3071.0,26,1,175.5,850.5,...,30,4,6,2,332.692308,389.538462,30.307692,53.215868,83.626926,-321.730769
263,LIDC-IDRI-0377,Node_N1,2,-882.321409,-3024.0,3071.0,2402,1,-307.0,61.0,...,169,29,24,9,382.402998,308.854288,173.039550,92.739302,28.898399,-68.460564
44,LIDC-IDRI-0167,Node_N1,1,-664.766231,-2048.0,3071.0,56,1,-444.5,-66.5,...,50,6,9,2,70.267857,174.964286,50.321429,-136.237780,-53.812866,-234.696429
219,LIDC-IDRI-0272,Node_N1,3,-824.358062,-2048.0,3071.0,51,1,-447.0,102.0,...,81,6,7,2,209.313725,390.941176,81.568627,-47.673652,80.722794,-109.078431
143,LIDC-IDRI-0234,Node_N1,1,-708.012378,-2048.0,3029.0,251,1,-569.0,82.0,...,41,11,14,3,367.756972,310.848606,41.689243,65.179121,43.765426,-236.276892


Scaling the data

In [87]:
# if DF_Radiomics_noduls_with_diagnose_train_data_scaled.csv and DF_Radiomics_noduls_with_diagnose_test_data_scaled.csv already exist, skip this step
# otherwise scale the data and save it to CSV files
if os.path.isfile('DF_Radiomics_noduls_with_diagnose_train_data_scaled.csv') and os.path.isfile('DF_Radiomics_noduls_with_diagnose_test_data_scaled.csv'):
    print("Scaled data already exists")
else:
    from sklearn.preprocessing import StandardScaler

    # Get all column names
    all_columns = train_data.columns.tolist()

    # Exclude the first three columns
    features = all_columns[3:]

    # Create a stratified split
    train_data, test_data = train_test_split(data, test_size=0.2, stratify=data['Labels'])

    # Create a scaler
    scaler = StandardScaler()

    # Fit the scaler on the training data and transform both training and test data
    train_data[features] = scaler.fit_transform(train_data[features])
    test_data[features] = scaler.transform(test_data[features])

    # Save the data to CSV files
    train_data.to_csv('DF_Radiomics_noduls_with_diagnose_train_data_scaled.csv', index=False)
    test_data.to_csv('DF_Radiomics_noduls_with_diagnose_test_data_scaled.csv', index=False)


Scaled data already exists


# Fully Connected Neural Network

In [88]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Assuming 'data' is your pandas DataFrame
# Ensure the DataFrame only contains numeric values
data = pd.read_csv('DF_Radiomics_noduls_with_diagnose_train_data_scaled.csv')
#drop patient and node columns
data = data.drop(['Patient', 'Node'], axis=1)
# TODO maybe add the columns later to see if it helps

Trainset

In [89]:
# Split data into features and labels
X = data.drop('Labels', axis=1).values
y = data['Labels'].values

# Convert to PyTorch tensors
X_tensor = torch.tensor(X).float()
y_tensor = torch.tensor(y).float()

# Stratified split
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, stratify=y_tensor, random_state=ran_seed)

# Create TensorDatasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

In [91]:
print("Train data:", train_dataset.tensors[0].shape)

Train data: torch.Size([197, 115])


In [92]:
# Model
class FCNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FCNN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout for regularization
            nn.Linear(hidden_size, hidden_size*2),
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout for regularization
            nn.Linear(hidden_size*2, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        return self.net(x)

# Hyperparameters
input_size = train_dataset.tensors[0].shape[1]  # Get the number of features from your dataset
hidden_size = input_size*2  # You can tune this
output_size = 4   # 3 labels 
learning_rate = 0.001
batch_size = 32
epochs = 50  # Adjust based on your runtime requirement
early_stopping_factor = 10
clip_value = 1  # for gradient clipping

# Initialize model, loss function, and optimizer
model = FCNN(input_size, hidden_size, output_size).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)  # L2 regularization

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [93]:
#check if cuda is available, print the gpu model name
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == "cuda":
    print(torch.cuda.get_device_name(0))
    # Move model to the device
    model = model.to(device)

NVIDIA GeForce RTX 3090


In [94]:
# # Initialize previous loss to infinity for comparison in the first epoch
# prev_loss = float('inf')
# stop_training = False

# # Training loop
# model.fc = nn.Linear(input_size, output_size)  # 'num_output_neurons' is the number of output neurons in your linear layer

# # Training loop
# model.train()
# for epoch in range(epochs):
#     for inputs, targets in train_loader:
    
#         # Check for nan or inf values in inputs
#         assert not torch.isnan(inputs).any(), "NaN values found in inputs"
#         assert not torch.isinf(inputs).any(), "Infinite values found in inputs"
        
#         # Move inputs and targets to the device
#         inputs = inputs.to(device)
#         targets = targets.to(device)
        
#         targets = targets.long()
        

#         # Zero the gradients
#         optimizer.zero_grad()

#         # Forward pass
#         outputs = model(inputs)

#         # Compute loss
#         loss = criterion(outputs, targets)
        
#         #naive early stopping
#         # If the loss has increased substantially, stop training
#         if loss.item() > prev_loss * early_stopping_factor:  # Adjust this factor as needed
#             print(f"Stopping training at epoch {epoch+1}/{epochs}, Loss: {loss.item()}")
#             stop_training = True
#             break
        

#         # Backward pass and optimize
#         loss.backward()
#         optimizer.step()
        
#         # Update previous loss
#         prev_loss = loss.item()
    
#     if stop_training:
#         break
    

    
#     print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

In [98]:
# Initialize best loss to infinity for comparison in the first epoch
best_loss = float('inf')

# Patience counter
patience_counter = 0

# Patience limit
patience_limit = 5

# Training loop
model.train()
for epoch in range(epochs):
    epoch_loss = 0
    for inputs, targets in train_loader:
    
        # Move inputs and targets to the device
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        targets = targets.long()

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, targets)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    # Average epoch loss
    epoch_loss = epoch_loss / len(train_loader)

    # If the training loss has improved, save the model and reset the patience counter
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        patience_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        # If the training loss has not improved, increment the patience counter
        patience_counter += 1
        if patience_counter >= patience_limit:
            print(f"Early stopping at epoch {epoch+1}/{epochs}, best loss: {best_loss}")
            break

    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss}")

# Load the best model
model.load_state_dict(torch.load('best_model.pth'))

Epoch 1/50, Loss: 0.15155739656516484
Epoch 2/50, Loss: 0.1835748212678092
Epoch 3/50, Loss: 0.20240314890231406
Epoch 4/50, Loss: 0.12604104966989585
Epoch 5/50, Loss: 0.09984725101717881
Epoch 6/50, Loss: 0.12222789919802121
Epoch 7/50, Loss: 0.1290846403156008
Epoch 8/50, Loss: 0.14346522944314138
Epoch 9/50, Loss: 0.12562836653419904
Early stopping at epoch 10/50, best loss: 0.09984725101717881


<All keys matched successfully>

# Evaluate
## Testset

In [102]:
# Evaluate the model
# Evaluation function
def evaluate(model, data_loader, device):
    model.eval()  # Set the model to evaluation mode
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():  # Disable gradient calculations
        for inputs, targets in data_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)  # Get the index of the max log-probability

            total_predictions += targets.size(0)
            correct_predictions += (predicted == targets).sum().item()

    accuracy = correct_predictions / total_predictions
    return accuracy

# Use the function
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
accuracy = evaluate(model, test_loader, device)
print(f'Accuracy: {accuracy * 100}%')

Accuracy: 84.0%


## Validation Set

In [101]:
validation_data = pd.read_csv('DF_Radiomics_noduls_with_diagnose_test_data_scaled.csv')

#create the tensor dataset
X_test = validation_data.drop(['Patient', 'Node', 'Labels'], axis=1).values
y_test = validation_data['Labels'].values
validation_dataset = TensorDataset(torch.tensor(X_test).float(), torch.tensor(y_test).float())

# Use the function
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
accuracy = evaluate(model, validation_loader, device)
print(f'Accuracy: {accuracy * 100}%')

Accuracy: 87.09677419354838%
